In [ ]:
import MDAnalysis as mda
import MDAnalysis.transformations as trans
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import font_manager
import scipy.constants as constants
from MDAnalysis.analysis import lineardensity as lin

# read coordinate
u = mda.Universe('../solution/P-SO2/npt_1.tpr', '../solution/P-SO2/solution.gro')
# select molecules 
molecule = u.select_atoms(f'moltype P-SO2')
water = u.select_atoms('moltype H2O')
AA = u.select_atoms('moltype DON')
NMP = u.select_atoms('moltype NMP')

solution = water + AA + NMP
#calculate density along z-axis
polymer_density = lin.LinearDensity(molecule, grouping='atoms').run()
water_density = lin.LinearDensity(water, grouping='atoms').run()
AA_density = lin.LinearDensity(AA, grouping='atoms').run()
NMP_density = lin.LinearDensity(NMP, grouping='atoms').run()

z_length = u.dimensions[2]
z_bins = len(polymer_density.results['z']['mass_density'])
z_range = np.linspace(0, z_length, z_bins)

plt.figure(figsize=(10, 6))
plt.plot(z_range, polymer_density.results['z']['mass_density'], label = 'Polymer')
plt.plot(z_range, water_density.results['z']['mass_density'], label = 'Water', color='steelblue')
plt.plot(z_range, AA_density.results['z']['mass_density'], label = 'AA', color = 'indianred')
plt.plot(z_range, NMP_density.results['z']['mass_density'], label = 'NMP', color = 'sandybrown')
plt.xlabel('Z-axis (Å)', fontsize=14)
plt.ylabel('Density (g/$nm^3$)', fontsize=14)
plt.title('Density profile', fontsize=16)
plt.xlim(0,u.dimensions[2])
plt.grid(alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
from scipy.ndimage import gaussian_filter1d
import numpy as np
import matplotlib.pyplot as plt

# smooth density curve
sigma = 4
y_H2O_smooth = gaussian_filter1d(water_density.results['z']['mass_density'], sigma)
y_polymer_smooth = gaussian_filter1d(polymer_density.results['z']['mass_density'], sigma)
y_AA_smooth = gaussian_filter1d(AA_density.results['z']['mass_density'], sigma)
y_NMP_smooth = gaussian_filter1d(NMP_density.results['z']['mass_density'], sigma)

# decide interface position using smoothed curve
threshold = max(y_polymer_smooth) * 0.9
threshold_2 = max(y_H2O_smooth) * 0.9
indices = np.where(y_polymer_smooth >= threshold)[0]

z_min_index = indices[0]
z_max_index = indices[-1]

left_index = z_min_index
while left_index > 0 and y_H2O_smooth[left_index] < threshold_2:
    left_index -= 1
right_index = z_max_index
while right_index < len(z_range) - 1 and y_H2O_smooth[right_index] < threshold_2:
    right_index += 1 

z_min_h2o = z_range[left_index]
z_max_h2o = z_range[right_index]
z_min = z_range[z_min_index]
z_max = z_range[z_max_index]
center = (z_min + z_max) / 2

xlim_min = max(0, center - 200)
xlim_max = min(400, center + 200)

print(f"z_min: {z_min:.2f} Å, z_max: {z_max:.2f} Å")

print(f'box edge : {z_min:.2f} to {z_max:.2f}')
print(f'water edge : {z_min_h2o:.2f} to {z_max_h2o:.2f}')


In [ ]:
# plot density profile
import matplotlib.pyplot as plt
plt.rcParams['axes.linewidth'] = 6
plt.rcParams['font.size'] = 14
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.weight'] = 'bold'
fig, ax = plt.subplots(figsize=(11,8))

# plot density profile
ax.plot(z_range, y_polymer_smooth, label='Polymer', lw=3, color='gray')
ax.plot(z_range, y_H2O_smooth, label='Water', lw=3, color='#2e4057')
ax.plot(z_range, y_AA_smooth, label='AA', lw=3, color='#048ba8')
ax.plot(z_range, y_NMP_smooth, label='NMP', lw=3, color='#f18f01')

# plot interface position
ax.axvline(z_min, color='red', linestyle='--', lw=2)  # Polymer min
ax.axvline(z_max, color='red', linestyle='--', lw=2)  # Polymer max
ax.axvline(z_min_h2o, color='red', linestyle='--', lw=2)  # Water min
ax.axvline(z_max_h2o, color='red', linestyle='--', lw=2)  # Water max.

# plot label
ax.set_xlabel('Z-axis (Å)', fontsize=32, fontweight='bold')
ax.set_ylabel(r'Density ($\rm g/nm^3$)', fontsize=32, fontweight='bold')
ax.set_xlim(0, 350)
ax.tick_params(axis='x', labelsize=22, width=3, length=8)
ax.tick_params(axis='y', labelsize=22, width=3, length=5)
ax.legend(fontsize=22, loc='upper right', frameon=False,bbox_to_anchor=(1.02, 1))
fig.tight_layout()
plt.show()
